In [1]:
import numpy as np
import pandas as pd
import xlearn as xl

In [2]:
# Data not added to GitHub. Any data will suffice
table = pd.read_pickle("../data/Day1")

In [3]:
filtered = table[(~np.isnan(table["c_cnt"]))]

In [4]:
filtered.c_cnt.value_counts()

0.0    3798680
1.0       2399
Name: c_cnt, dtype: int64

In [5]:
CLICKED_CNT = filtered.c_cnt.value_counts()[1]

## Attempting to convert data to FFM format

#### Note: unfinished, but concepts are covered in comments until end of section

In [13]:
from sklearn.utils import shuffle
filtered = shuffle(filtered)
small_filtered = filtered.head(10000)

In [24]:
filtered.columns

Index(['ad_network_id', 'ad_type', 'advertiser_id', 'bid_requests',
       'bid_responses', 'c_cnt', 'c_timestamp', 'c_txn_fee', 'c_txn_rate',
       'campaign_id', 'campaign_type', 'cr_cnt', 'creative_id', 'exp_mode',
       'f_cnt', 'geo_continent_code', 'geo_country_code2', 'geo_dma_code',
       'geo_region_name', 'geo_timezone', 'i_cnt', 'i_timestamp',
       'pub_network_id', 'r_cnt', 'r_num_ads_requested', 'r_num_ads_returned',
       'r_num_ads_third_party', 'r_timestamp', 'rate_metric', 'session_id',
       'site_id', 'token', 'txn_fee', 'txn_rate', 'ua_device',
       'ua_device_type', 'ua_name', 'ua_os_name', 'vi_cnt', 'vi_timestamp',
       'vv_cnt', 'zone_id'],
      dtype='object')

In [29]:
ctr_irrelevant = ['c_timestamp', 'c_txn_fee', 'c_txn_rate', 'session_id', 'txn_fee', 'txn_rate']
new_cols = [col for col in small_filtered.columns if col not in ctr_irrelevant]
small_filtered = small_filtered[new_cols]

In [30]:
# Getting num of unique elements (out of 10000) in each column to determine whether we need the column
# If only one element in a column, we can delete it (it doesn't provide any new information)
# We can also use this info later to figure out how to discretize numerical data

# aggregating columns that don't provide information i.e. they're all the same value
useless = []

for col in small_filtered.columns:
    try:
        print(col, end=": ")
        numelems = len(set(small_filtered[col]))
        print(numelems)
        if numelems == 1:
            useless.append(col)
    except TypeError as e:
        print(e)

ad_network_id: 18
ad_type: unhashable type: 'list'
advertiser_id: 22
bid_requests: unhashable type: 'list'
bid_responses: unhashable type: 'list'
c_cnt: 2
campaign_id: 34
campaign_type: 2
cr_cnt: 1
creative_id: 87
exp_mode: 9525
f_cnt: 1
geo_continent_code: 6
geo_country_code2: 70
geo_dma_code: 206
geo_region_name: 342
geo_timezone: 235
i_cnt: 2
i_timestamp: 9999
pub_network_id: 2
r_cnt: 2
r_num_ads_requested: 6
r_num_ads_returned: 8
r_num_ads_third_party: 1
r_timestamp: 10000
rate_metric: 1
site_id: 12
token: 87
ua_device: 571
ua_device_type: 38
ua_name: 39
ua_os_name: 18
vi_cnt: 2
vi_timestamp: 10000
vv_cnt: 1
zone_id: 20


In [31]:
useless

['cr_cnt', 'f_cnt', 'r_num_ads_third_party', 'rate_metric', 'vv_cnt']

In [35]:
new_cols = [col for col in small_filtered.columns if col not in useless]
new_cols.remove('c_cnt') # this is our flag; it needs to be placed at the front
new_cols = ['c_cnt'] + new_cols
refiltered = filtered[new_cols]

In [36]:
refiltered.shape

(3801079, 31)

In [37]:
# Here we map each feature to a representative number
# Right now we're working with 37 features
tuples = zip(refiltered.columns, range(len(refiltered.columns)))
featuremap = dict(tuples)
len(featuremap)

31

In [ ]:
# We manipulate each feature accordingly
# Categorical features we leave alone if already integral
# Non-integral categorical features we map to integers
# Numerical features we discretize with ranges
categorical = ['ad_network_id', 'ad_type', 'advertiser_id', 'bid_requests', ]

## End of section

In [ ]:
from sklearn.utils import shuffle
filtered = shuffle(filtered)

In [ ]:
def generate_sample(n):
    """Return X matrix with n no click samples (undersampling)"""
    clicked = filtered[filtered["c_cnt"] == 1]
    no_click = filtered[filtered["c_cnt"] == 0].sample(n)
    return shuffle(pd.concat([clicked, no_click]))

In [ ]:
# Create X and Y matrix for model
sample = generate_sample(CLICKED_CNT + 200)
sample.dropna(subset=["ua_device_type"], inplace=True)
sample.dropna(subset=["i_timestamp"], inplace=True)

In [ ]:
sample.shape

In [ ]:
import xlearn as xl

In [ ]:
ffm_model = xl.create_ffm()
ffm_model.setTrain(sample)